CORRER ESTA CELDA PARA CONECTAR A DRIVE
* elegir cuenta
* poner permitir
* copiar el token y pegar al input


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 


Mounted at /content/gdrive/


CALLBACKS




In [ ]:
from keras.callbacks import Callback
import pickle
from datetime import datetime
import time

class MyCheckpoint(Callback):
  def __init__(self, start_epoch, exp_folder, weights_path, pickles_path, EXP_NAME, model_name):
    super(MyCheckpoint, self).__init__()
    self.current_epoch = start_epoch + 1  
    self.timestamps = {}
    self.exp_folder = exp_folder
    self.weights_path = weights_path
    self.pickles_path = pickles_path
    self.EXP_NAME = EXP_NAME  
    self.model_name = model_name
    self.model_training_time = 0
    self.start_batch = 0
    self.end_batch = 0


  def on_epoch_begin(self, epoch, logs=None):
    self.timestamps[self.current_epoch] = {}   
    self.timestamps[self.current_epoch]["start"] = datetime.now().time().strftime("%H:%M")  

  def on_test_begin(self, logs=None):    
    self.model.save_weights(os.path.join(self.weights_path+'{}_epoch_{}.h5'.format(self.model_name,self.current_epoch)))          
    self.timestamps[self.current_epoch]["end"] = datetime.now().time().strftime("%H:%M")  # PASAR ARRIBA POR EL SAVE
    self.timestamps[self.current_epoch]["train_time"] = self.model_training_time 
    with open(os.path.join(self.exp_folder, f"timestamps_epoch{self.current_epoch}.pickle"), 'wb') as fd:
      pickle.dump(self.timestamps, fd)
    self.model_training_time = 0

  def on_train_batch_begin(self, batch, logs=None):
      self.start_batch = time.time()

  def on_train_batch_end(self, batch, logs=None):
      self.end_batch = time.time()
      self.model_training_time+= (self.end_batch - self.start_batch)

  def on_epoch_end(self, epoch, logs=None):            
    with open(os.path.join(self.pickles_path+'{}_epoch_{}.pickle'.format(self.model_name,self.current_epoch)), 'wb') as fd:
      pickle.dump(logs, fd)
    self.current_epoch = self.current_epoch + 1    


MUY IMPORTANTE EL FOLD QUE ELIJAS ACA TIENE QUE COINCIDIR CON LA CARPETA QUE ELIJAS DESPUES

In [ ]:
import os
#Esto es lo que cambia
FOLD_NAME = '0fold'

#Esto es fijo
DATASET = 'Multisampling16'
MODEL_NAME = 'rgb'+FOLD_NAME

basePath = "/content/gdrive/My Drive/Informática/Tesina/"
videos_path = basePath+'Videos/'+DATASET
fls = os.listdir(videos_path)

OBJETO EXPERIMENTO

In [ ]:
import os

from keras.initializers import HeNormal
from keras.regularizers import l1_l2
from keras.activations import elu
from keras.callbacks import TensorBoard, EarlyStopping

basePath = "/content/gdrive/My Drive/Informática/Tesina/"

datasets = {            
            "Multisampling32":{"path": basePath+"Videos/Multisampling32/","num_frames": 32,"files_posfix":"Multisampling32"},  
            "Multisampling16":{"path": basePath+"Videos/Multisampling16/","num_frames": 16, "files_posfix":"Multisampling16"},  
            "Multisampling8":{"path": basePath+"Videos/Multisampling8/","num_frames": 8, "files_posfix":"Multisampling8"},  
            "SimpleSampling32":{"path": basePath+"Videos/SimpleSampling32/","num_frames": 32, "files_posfix":"SimpleSampling32"},              
  }

class Experiment():

  def __init__(self, exp_name, model_name):
    #Logging and checkpoints
    self.EXP_NAME = exp_name
    self.model_name = model_name
    self.exp_folder = os.path.join(basePath+"Experimentos/Conv3D", exp_name, model_name)
    self.weights_path = os.path.join(self.exp_folder,"Weights/")
    self.pickles_path = os.path.join(self.exp_folder,"Pickles/")
    if (os.path.exists(self.exp_folder)):           
      weights = os.listdir(self.weights_path)
      # get the latest epoch trained
      try:
        self.current_epoch = sorted(list(map(lambda x: int(x.split('_')[-1].split('.')[0]), weights)), reverse=True)[0] 
      except (TypeError, IndexError):
        self.current_epoch = 0      
    else:      
      os.mkdir(self.exp_folder)      
      os.mkdir(self.weights_path)
      os.mkdir(self.pickles_path)
      self.current_epoch = 0

    tensorboard = TensorBoard(log_dir=os.path.join(self.exp_folder, 'logs'), histogram_freq=1)
    early = EarlyStopping(monitor='val_accuracy', patience=12)
    self.testing = False
    #Data specs
    data_dict = datasets[DATASET]
    self.fold_name = FOLD_NAME
    self.files_posfix = DATASET
    self.names_path = os.path.join(basePath+"/DatasetsNames", DATASET)
    self.videos_path = data_dict['path']
    self.num_frames = data_dict['num_frames']
    self.height = 128 #data_dict['height']
    self.width = 128 #data_dict['width']
    self.channels = 1
    #Model specs
    self.activation = elu
    self.initializer = HeNormal
    self.l1=1e-5
    self.l2=1e-5
    self.regularizer = l1_l2(l1=self.l1, l2=self.l2)
    self.conv_blocks = [
                        {'filters':16, 'kernel_size':3, 'pool_size':3},
                        {'filters':32, 'kernel_size':3, 'pool_size':3},
                        {'filters':32, 'kernel_size':3, 'pool_size':3},  
                        {'filters':64, 'kernel_size':3, 'pool_size':3},  
                    ]


    self.batch_size = 32
    self.epochs_to_train = 20-self.current_epoch
    self.lr = 0.001
    self.callbacks = [MyCheckpoint(self.current_epoch, self.exp_folder, self.weights_path, self.pickles_path, self.EXP_NAME, self.model_name), tensorboard, early]
    
      

  def save_resume(self):
    with open(os.path.join(self.exp_folder,'exp_resume.txt'), 'w') as f:
      stri = ""
      settings = [a for a in dir(self) if not a.startswith('__')]
      for setting in settings:
        stri = stri + setting+": "+str(getattr(experiment, setting))+'\n'
      f.writelines(stri)

MODELOS


In [ ]:

from keras.layers import Input, Conv3D, BatchNormalization, MaxPooling3D, Dense, GlobalAveragePooling3D
from keras import Model
from keras.initializers import HeNormal
from keras.regularizers import l2


def conv_3d_block(input,index,filters, kernel_size, activation,initializer,regularizer, pool_size):
  conv = Conv3D(
      filters=filters, 
      kernel_size=(kernel_size,kernel_size, kernel_size),      
      activation=activation, 
      kernel_initializer=initializer(),
      kernel_regularizer=regularizer, 
      padding='same',      
      name=f"Conv_{index}")(input)
  batch = BatchNormalization(name=f"Batch_{index}")(conv)
  pooling = MaxPooling3D(pool_size=(pool_size,pool_size,pool_size), padding='same',name=f"3dPool_{index}")(batch)  
  return pooling

def last_conv_3d_block(input,filters, kernel_size, activation,initializer,regularizer, pool_size):
  conv = Conv3D(
      filters=filters, 
      kernel_size=(kernel_size,kernel_size, kernel_size),      
      activation=activation, 
      kernel_initializer=initializer(),
      kernel_regularizer=regularizer, 
      padding='same',      
      name="final_conv")(input) #La proxima entonces toma solo lo ultimo
  batch = BatchNormalization(name="final_batch")(conv)
  avg_pooling = GlobalAveragePooling3D()(batch)  
  return avg_pooling

def conv_stack(input, experiment):
  stack = input
  for i in range(0,len(experiment.conv_blocks)-1):
    stack = conv_3d_block(stack,i,
      experiment.conv_blocks[i]['filters'],
      experiment.conv_blocks[i]['kernel_size'],
      experiment.activation,
      experiment.initializer,
      experiment.regularizer,
      experiment.conv_blocks[i]['pool_size']
    )
  stack = last_conv_3d_block(stack,
      experiment.conv_blocks[-1]['filters'],
      experiment.conv_blocks[-1]['kernel_size'],
      experiment.activation,
      experiment.initializer,
      experiment.regularizer,
      experiment.conv_blocks[-1]['pool_size']
  )
  return stack

def Conv3D_model(experiment,categories=64):
  input = Input(shape=(experiment.num_frames, experiment.height, experiment.width,experiment.channels))
  stack = conv_stack(input, experiment)
  output = Dense(categories, activation='softmax')(stack)
  model = Model(inputs=input, outputs=output, name=experiment.model_name)  
  return model

GENERADOR

In [ ]:
import cv2
import imageio
import numpy as np
from keras.utils import to_categorical
def generator_cropped(videos_path,samples, batch_size, categories=64):
  """
    samples serian los nombres de archivos
  """
  height_upper_limit = int(1080/4)
  width_upper_limit = int(1920/4)
  height_to_crop = int(height_upper_limit*3)
  width_to_crop = int(width_upper_limit*3)
  w = experiment.width
  h = experiment.height
  num_samples = len(samples) 
  while True:
    for offset in range(0,num_samples, batch_size):
      batch_samples = samples[offset:offset+batch_size]
      X_train = []
      y_train = []
      for batch_sample in batch_samples:
        fname, w_limit, h_limit = batch_sample.split(" ")
        w_limit, h_limit = int(w_limit), int(h_limit)
        vidcap = cv2.VideoCapture(videos_path+fname)      
        #vidcap = imageio.get_reader(videos_path+batch_sample)
        frames = []
        suc, frame = vidcap.read()                
        #for i, frame in enumerate(vidcap):
        while suc: 
          frame = frame[h_limit:h_limit+height_to_crop,w_limit:w_limit+width_to_crop,:]
          frame = cv2.resize(frame, (w,h), interpolation=cv2.INTER_AREA)
          frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if (experiment.channels==3) else np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),2)
          frame = frame.astype(np.float32)
          frame /= 255.0          

          frames.append(frame)
          suc, frame = vidcap.read()                
        X_train.append(frames)        
        cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
        y_train.append(to_categorical(cat, num_classes=categories))      
      try:
        X_train = np.asarray(X_train, dtype=np.float32)      
      except:
        print("Trying again...")
        try:
          X_train = np.asarray(X_train, dtype=np.float32)
        except:
          print("failed reading")      
      y_train = np.asarray(y_train)      
      yield X_train, y_train

In [ ]:
import cv2
import imageio
import numpy as np
from keras.utils import to_categorical
def generator(videos_path,samples, batch_size, categories=64):
  """
    samples serian los nombres de archivos
  """
  w = experiment.width
  h = experiment.height
  num_samples = len(samples) 
  while True:
    for offset in range(0,num_samples, batch_size):
      batch_samples = samples[offset:offset+batch_size]
      X_train = []
      y_train = []
      for batch_sample in batch_samples:
        vidcap = cv2.VideoCapture(videos_path+batch_sample)      
        #vidcap = imageio.get_reader(videos_path+batch_sample)
        frames = []
        suc, frame = vidcap.read()                
        #for i, frame in enumerate(vidcap):
        while suc: 
          frame = cv2.resize(frame, (w,h), interpolation=cv2.INTER_AREA)
          
          
          frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if (experiment.channels==3) else np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),2)
          
          
          frame = frame.astype(np.float32)
          
          
          frame /= 255.0          
          
          

          frames.append(frame)
          suc, frame = vidcap.read()        
        
        X_train.append(frames)        
        cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
        y_train.append(to_categorical(cat, num_classes=categories))      
      try:
        X_train = np.asarray(X_train, dtype=np.float32)      
      except:
        print("Trying again...")
        try:
          X_train = np.asarray(X_train, dtype=np.float32)
        except:
          print("failed reading")      
      y_train = np.asarray(y_train)      
      yield X_train, y_train


In [ ]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical
from keras.applications import mobilenet
from keras.utils import Sequence
import math 
class SequenceGenerator(Sequence):
  def __init__(self, videos_path, samples,batch_size, width, height, channels):
    self.batch_size = batch_size
    self.videos_path = videos_path
    self.samples = samples
    self.width = width
    self.height = height
    self.channels = channels
    self.len = len(os.listdir(videos_path))

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    batch_samples = self.samples[idx * self.batch_size:(idx+1)*self.batch_size]
    X_train = []
    y_train = []
    for batch_sample in batch_samples:      
      vidcap = cv2.VideoCapture(os.path.join(self.videos_path,batch_sample))              
      frames = []
      suc, frame = vidcap.read()              
      while suc: 
        frame = cv2.resize(frame, (self.width,self.height), interpolation=cv2.INTER_AREA)
        frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if (self.channels==3) else np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),2)
        frame = frame.astype(np.float32)
        frame /= 255.0          
        frames.append(frame)
        suc, frame = vidcap.read()      
      
      X_train.append(frames)        
      cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
      y_train.append(to_categorical(cat, num_classes=64))      
        
    X_train = np.asarray(X_train, dtype=np.float32)      
    y_train = np.asarray(y_train)      
    return X_train, y_train

INICIALIZACION:


*   CORRES LA CELDA
*   ELEGIS LA OPCION NRO 3 (MULTISAMPLING32)
*   SI ESTAS USANDO UN FOLD NUEVO PONES: y
  * ESCRIBIS 'rgb{FOLDNAME}fold' [i.e 'rgb2fold']
*   SI ESTAS UN FOLD QUE YA EMPEZASTE A ENTRENAR PONES: n
  * ELEGIS EL NUMERO DE FOLD QUE ESTES ENTRENANDO



In [ ]:
from keras.optimizers import Adam
from keras.models import load_model
datasetsFolders = os.listdir("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D")
for d in datasetsFolders:
  print(f"{datasetsFolders.index(d)} - {d}")
print("Ingrese el numero del Dataset: ")
#chosen = input()
#exp_name = datasetsFolders[int(chosen)]
print(DATASET)
exp_name = DATASET
#if not os.path.exists(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D", )):
#  os.mkdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D", exp_name))
print("¿Nuevo experimento? [y|n]" )
print('y')
print('Model name: '+MODEL_NAME)
#if input()=='y':
#  print("Experimentos Previos:")
#  print(os.listdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D", exp_name)))
#  print("Nombre de modelo [default: Baseline]: ")
#  model_name = input()
#  model_name = "baseline" if model_name == '' else model_name
#  if model_name in os.listdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D", exp_name)):
#    print("Ya esta el experimento, empiece de vuelta")
#    raise Exception
#    
#else:
#  experiments = os.listdir(os.path.join("/content/gdrive/My Drive/Informática/Tesina/Experimentos/Conv3D", exp_name))
#  for d in experiments:
#    print(f"{experiments.index(d)} - {d}")
#  print("Ingrese el numero del Modelo: ")
#  chosen = input()  
#  model_name = experiments[int(chosen)]
  


experiment = Experiment(exp_name, MODEL_NAME)
experiment.save_resume()
weights_path = experiment.weights_path
pickles_path = experiment.pickles_path
videos_path = experiment.videos_path

0 - Brasil
1 - Multisampling16
2 - Multisampling8
3 - Multisampling32
4 - SimpleSampling32
Ingrese el numero del Dataset: 
SimpleSampling32
¿Nuevo experimento? [y|n]
y
Model name: gray0fold


TRAIN VAL TEST SPLIT


In [ ]:

with open(os.path.join(experiment.names_path,f'train_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  train = f.read().split('\n')
with open(os.path.join(experiment.names_path,f'val_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  val = f.read().split('\n')
with open(os.path.join(experiment.names_path,f'test_{experiment.fold_name}_{experiment.files_posfix}.txt'), 'r') as f:
  test = f.read().split('\n')


ENTRENAMIENTO
CORRER ESTA CELDA 2 VECES:
* PONES A CORRER
* FALLA
* VOLVER A CORRER

In [ ]:

batch_size = experiment.batch_size

if f"{experiment.model_name}.pickle" in os.listdir(experiment.exp_folder):  
  with open(os.path.join(experiment.exp_folder, f"{experiment.model_name}.pickle"), 'rb') as pklfile:
    conf = pickle.load(pklfile)    
    model = Model.from_config(conf)
else:
  model = Conv3D_model(experiment)
  with open(os.path.join(experiment.exp_folder, f"{experiment.model_name}.pickle"), 'wb') as pklfile:
    conf = model.get_config()
    pickle.dump(conf, pklfile)
    with open(os.path.join(experiment.exp_folder, f'{experiment.model_name}.txt'), 'w') as f:
      model.summary(print_fn=lambda x: f.write(x + '\n'))
model.summary()


if '{}_epoch_{}.h5'.format(experiment.model_name, experiment.current_epoch) in os.listdir(weights_path):
  model.load_weights(os.path.join(experiment.weights_path,'{}_epoch_{}.h5'.format(experiment.model_name, experiment.current_epoch)))

adam = Adam(learning_rate=experiment.lr)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


if experiment.testing:
  print("Estas seguro que queres testear? [y/n]")
  if (input() == 'y'):
    eval_dict = model.evaluate(
        generator(experiment.videos_path,test, batch_size)
      , steps=int(len(test)/batch_size)
      , batch_size=experiment.batch_size
      , verbose = 1
      , return_dict=True
    )
    with open(os.path.join(experiment.exp_folder, f'evaluation_epoch{experiment.current_epoch}_toLength.pickle'), 'wb') as pkl:
      pickle.dump(eval_dict, pkl)
  else: 
    print("Cambia el experimento entonces en experiment.testing")
else:
  train_gen = SequenceGenerator(videos_path, train, batch_size, experiment.width, experiment.height, experiment.channels)    
  val_gen = SequenceGenerator(videos_path, val, batch_size, experiment.width, experiment.height, experiment.channels)    
  history = model.fit(
    train_gen
    , steps_per_epoch=int(len(train)/batch_size)
    , validation_data = val_gen
    ,validation_steps = int(len(val)/batch_size)
    , epochs = experiment.epochs_to_train
    , verbose = 1
    , shuffle = False    
    , callbacks=list(experiment.callbacks)
    , use_multiprocessing=True
    , workers=4
    )










Model: "noreg0fold"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 64, 64, 1)]   0         
_________________________________________________________________
Conv_0 (Conv3D)              (None, 32, 64, 64, 16)    448       
_________________________________________________________________
Batch_0 (BatchNormalization) (None, 32, 64, 64, 16)    64        
_________________________________________________________________
3dPool_0 (MaxPooling3D)      (None, 16, 32, 32, 16)    0         
_________________________________________________________________
Conv_1 (Conv3D)              (None, 16, 32, 32, 32)    13856     
_________________________________________________________________
Batch_1 (BatchNormalization) (None, 16, 32, 32, 32)    128       
_________________________________________________________________
3dPool_1 (MaxPooling3D)      (None, 8, 16, 16, 32)     0

SI SE TRABA CORRER ESTO PARA SABER QUE HAY QUE HACER

In [ ]:
try:
  print("Actualizando epoca actual...")
  print('Previo: '+str(experiment.current_epoch))
  weights = os.listdir(experiment.weights_path)
  experiment.current_epoch = sorted(list(map(lambda x: int(x.split('_')[-1].split('.')[0]), weights)), reverse=True)[0] 
  experiment.epochs_to_train = 20-experiment.current_epoch
  print('Actual: '+str(experiment.current_epoch))
  print('Epocas a entrenar: '+str(experiment.epochs_to_train))
  print('SE PUEDE VOLVER A EJECUTAR LA CELDA DE ENTRENAMIENTO!')
except:
  print("ERROR!\nCORRER DE VUELTA DESDE LA PRIMER CELDA (MONTADO DE DRIVE)")

Actualizando epoca actual...
ERROR!
CORRER DE VUELTA DESDE LA PRIMER CELDA (MONTADO DE DRIVE)


In [ ]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical
from keras.applications import mobilenet
from keras.utils import Sequence
import math 
class SequenceGenerator(Sequence):
  def __init__(self, videos_path, samples,batch_size, width, height, channels):
    self.batch_size = batch_size
    self.videos_path = videos_path
    self.samples = samples
    self.width = width
    self.height = height
    self.channels = channels
    self.len = len(os.listdir(videos_path))

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    batch_samples = self.samples[idx * self.batch_size:(idx+1)*self.batch_size]
    X_train = []
    y_train = []
    for batch_sample in batch_samples:      
      vidcap = cv2.VideoCapture(os.path.join(self.videos_path,batch_sample))              
      frames = []
      suc, frame = vidcap.read()              
      while suc: 
        frame = cv2.resize(frame, (self.width,self.height), interpolation=cv2.INTER_AREA)
        frame =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if (self.channels==3) else np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),2)
        frame = frame.astype(np.float32)
        frame /= 255.0          
        frames.append(frame)
        suc, frame = vidcap.read()      
      
      X_train.append(frames)        
      cat = int(batch_sample.split('_')[0])#  Los mp4 estan categorizados de 1 a 64 y los .avi de 0 a 64, asi que no necesito restar 1        
      y_train.append(to_categorical(cat, num_classes=64))      
        
    X_train = np.asarray(X_train, dtype=np.float32)      
    y_train = np.asarray(y_train)      
    return X_train, y_train
    

In [ ]:
DATASET = 'Multisampling8'
import numpy as np
import pickle
import functools as ft
from keras import Model
from keras.optimizers import Adam
def addHistories(h1, h2):    
    for key in ["accuracy", "loss", "val_loss", "val_accuracy"]:
        h2[key].append(h1[key]) #ya no es una lista asi que puedo acceder a la metrica directo 
    return h2

basePath = "/content/gdrive/My Drive/Informática/Tesina/"
baseFolder = basePath+'Experimentos/Conv3D/'+DATASET
namesPath = basePath+'DatasetsNames/'+DATASET
videos_path = basePath+'Videos/'+DATASET
best_epochs = {}
#mean_acc = {'rgb':[], 'rgb64_':[],'gray':[], 'gray64_':[]}
#mean_val_acc = {'rgb':[], 'rgb64_':[],'gray':[], 'gray64_':[]}
mean_acc = {'8':[], '16':[], '32':[]}
videos_path = {'8':basePath+'Videos/Multisampling8/','16':basePath+'Videos/Multisampling16/', '32':basePath+'Videos/Multisampling32/' }
#model_settings = {'rgb':{'res':128, 'colours':3}, 'rgb64_':{'res':64, 'colours':3},'gray':{'res':128, 'colours':1}, 'gray64_':{'res':64, 'colours':1}}

In [ ]:

for fold in ['0fold', '1fold', '2fold','3fold', '4fold']:
  #for model_type in ['rgb', 'rgb64_','gray', 'gray64_']:
    modelName = 'rgb'+fold
    expFolder = os.path.join(baseFolder,modelName)
    if f"{modelName}.pickle" in os.listdir(expFolder):  
      with open(os.path.join(expFolder, f"{modelName}.pickle"), 'rb') as pklfile:
        conf = pickle.load(pklfile)    
        model = Model.from_config(conf)
    else:
      print("no encuentro modelo")
    adam = Adam(learning_rate=0.001)
    model.compile(optimizer=adam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    EXP_PKL_PATH = os.path.join(expFolder, 'Pickles')  
    pkl_files = os.listdir(EXP_PKL_PATH)
    fls_s = sorted(pkl_files, key=lambda x: int(x.split('_')[-1].split('.')[0])) #Exp_18_epoch_2.pickle

    pkls = []
    for pkl_file in fls_s:
        with open(os.path.join(EXP_PKL_PATH ,pkl_file), 'rb') as pkl:
            h_temp = pickle.load(pkl)        
            #print(h_temp)
            if 'val_loss' in h_temp.keys():
              pkls.append(h_temp)


    init_dict = {}
    for k in ["accuracy", "loss", "val_loss", "val_accuracy"]:
      init_dict[k] = []
    hs = ft.reduce(lambda a,b: addHistories(b,a), pkls, init_dict)
    best_epoch = np.array(hs['val_accuracy']).argmax()+1
    best_epochs[fold] = (best_epoch,hs['val_accuracy'][best_epoch-1])
    #mean_val_acc[model_type].append((best_epoch,hs['val_accuracy'][best_epoch-1]))
    model.load_weights(os.path.join(expFolder,'Weights','{}_epoch_{}.h5'.format(modelName, best_epoch)))
    print(modelName)
    print(f"Entreno {len(hs['val_accuracy'])}")
    print((best_epoch,hs['val_accuracy'][best_epoch-1]))
    with open(os.path.join(namesPath,f'test_{fold}_{DATASET}.txt'), 'r') as f:
      test = f.read().split('\n')  
    for num_frames in ['8', '16', '32']:
      path = videos_path[num_frames]
      #gen = SequenceGenerator(path, test, 32, model_settings[model_type]['res'], model_settings[model_type]['res'], model_settings[model_type]['colours'])    
      gen = SequenceGenerator(path, test, 32, 128, 128, 3)    
      eval_dict = model.evaluate(
            gen
          , steps=int(len(test)/32)
          , batch_size=32
          , verbose = 1
          , return_dict=True
          ,use_multiprocessing=True
          ,workers=4
        )
      #mean_acc[model_type].append(eval_dict['accuracy'])
      mean_acc[num_frames].append(eval_dict['accuracy'])
      with open(os.path.join(expFolder, f'evaluation_epoch{best_epoch}_frames.pickle'), 'wb') as pkl:
        pickle.dump(eval_dict, pkl)    

with open(os.path.join(baseFolder, f'tests.pickle'), 'wb') as pkl:
      pickle.dump(mean_acc, pkl)

with open(os.path.join(baseFolder, f'bestEpochs.pickle'), 'wb') as pkl:
      pickle.dump(best_epochs, pkl)        
    

    

rgb0fold
Entreno 20
(18, 0.96875)
80/80 [==============================] - 1033s 13s/step - loss: 17.6184 - accuracy: 0.0996
rgb1fold
Entreno 20
(20, 0.981249988079071)
80/80 [==============================] - 1031s 13s/step - loss: 9.7188 - accuracy: 0.1668
rgb2fold
Entreno 20
(19, 0.9937499761581421)
80/80 [==============================] - 1035s 13s/step - loss: 32.5067 - accuracy: 0.0270
rgb3fold
Entreno 20
(20, 0.981249988079071)
80/80 [==============================] - 1031s 13s/step - loss: 80.3471 - accuracy: 0.0156
rgb4fold
Entreno 19
(17, 0.9156249761581421)
80/80 [==============================] - 1026s 13s/step - loss: 30.7516 - accuracy: 0.0371


In [ ]:
gen = SequenceGenerator(path, test, 32, 128, 128, 3)    
eval_dict = model.evaluate(
      gen
    , steps=int(len(test)/32)
    , batch_size=32
    , verbose = 1
    , return_dict=True
    ,use_multiprocessing=True
    ,workers=4
  )
#mean_acc[model_type].append(eval_dict['accuracy'])
mean_acc[num_frames].append(eval_dict['accuracy'])
with open(os.path.join(expFolder, f'evaluation_epoch{best_epoch}_frames.pickle'), 'wb') as pkl:
  pickle.dump(eval_dict, pkl)    

80/80 [==============================] - 993s 12s/step - loss: 0.0582 - accuracy: 1.0000


In [ ]:
mean_acc['8'].pop()

0.02187499962747097

In [ ]:
mean_acc

{'16': [0.2789062559604645, 0.3910156190395355, 0.42851561307907104],
 '32': [0.9984375238418579, 0.994921863079071, 1.0],
 '8': [0.03750000149011612, 0.02187499962747097, 0.05546874925494194]}

In [ ]:
with open(os.path.join(baseFolder, f'tests.pickle'), 'wb') as pkl:
      pickle.dump(mean_acc, pkl)

In [ ]:
with open(os.path.join(baseFolder, f'bestEpochs.pickle'), 'wb') as pkl:
      pickle.dump(best_epochs, pkl)   

In [ ]:
with open(os.path.join(expFolder, f'evaluation_epoch{best_epoch}_frames.pickle'), 'wb') as pkl:
      pickle.dump(eval_dict, pkl)    